In [1]:
import numpy as np
import torch
from tsGaussian.torch_tsgaussian import TangentSpaceGaussian
from stable_baselines_utils import TangentSpaceGaussian as TSG
# from pytorch3d.transforms.so3 import (
#     so3_exp_map,
#     so3_relative_angle,
# )

In [2]:
tg = TangentSpaceGaussian(None)

# Test liegroup torch

In [3]:
from liegroups.torch import SO3

In [4]:
C = SO3.exp(torch.Tensor([[1,2,3],
                         [0,0,0]]))
C

<liegroups.torch.so3.SO3Matrix>
| tensor([[[-0.6949,  0.7135,  0.0893],
|          [-0.1920, -0.3038,  0.9332],
|          [ 0.6930,  0.6313,  0.3481]],
| 
|         [[ 1.0000,  0.0000,  0.0000],
|          [ 0.0000,  1.0000,  0.0000],
|          [ 0.0000,  0.0000,  1.0000]]])

# Test torch_tsgaussian sample

In [5]:
R_mu = torch.eye(3).reshape((1,3,3))
sigma = torch.ones(3).reshape((1,3))

In [6]:
R_quat, R_x = tg.rsample(R_mu, sigma)

sigma:  tensor([[1., 1., 1.]])
torch.Size([1, 3, 3])


In [7]:
torch.bmm(torch.transpose(R_x, 1, 2), R_x)

tensor([[[ 1.0000e+00,  0.0000e+00, -4.4703e-08],
         [ 0.0000e+00,  1.0000e+00,  5.9605e-08],
         [-4.4703e-08,  5.9605e-08,  1.0000e+00]]])

# Test torch_tsgaussian normal_term

In [8]:
sigma = torch.ones(3).reshape((1,3))
sigma

tensor([[1., 1., 1.]])

In [9]:
tg.normal_term(sigma)

tensor([15.7496])

# Test torch_tsgaussian log_map

In [10]:
R_1 = torch.eye(3).reshape((1, 3, 3))
R_2 = torch.eye(3).reshape((1, 3, 3))

In [11]:
tg.log_map(R_1, R_2)

tensor([0., 0., 0.])

# Test torch_tsgaussian log_probs

In [12]:
R_x = torch.eye(3).reshape((1,3,3))
R_mu = torch.zeros(3,3).reshape((1,3,3))
R_x = R_x.repeat(5, 1, 1)
R_mu = R_mu.repeat(5, 1, 1)
sigma = torch.ones(3).reshape((1,3))

In [13]:
# tg.log_probs(R_x, R_mu, sigma)

In [14]:
np.e ** (-2.7568)

0.06349462641817973

all codes run for torch_tsgaussian now, need to check it's correctness and make it into batch version.

# Test TangentSpaceGaussian actions_from_params

In [15]:
tsg = TSG(None)

In [16]:
print(tsg.distribution)

In [17]:
tsg

In [18]:
tsg.actions_from_params(torch.eye(3).reshape((1,3,3)), torch.ones(3).reshape((1,3)))

sigma:  tensor([[1., 1., 1.]])
torch.Size([1, 3, 3])


(tensor([[0.5141, 0.2487, 0.0094, 0.8208]]),
 tensor([[[ 0.8762,  0.2403,  0.4179],
          [ 0.2711,  0.4712, -0.8393],
          [-0.3986,  0.8486,  0.3478]]]))

# Test TangentSpaceGaussian log_prob_from_params

In [19]:
torch.eye(3).repeat(2,1,1).size()

torch.Size([2, 3, 3])

In [20]:
torch.ones(3).repeat(2,1).size()

torch.Size([2, 3])

In [21]:
# tsg.log_prob_from_params(torch.eye(3).repeat(2,1,1), torch.ones(3))

In [22]:
x = torch.randn(2, 4, 4)
y = torch.linalg.inv(x)
y

tensor([[[ 1.5790e-01,  2.1328e-01,  3.6575e-01, -4.2418e-01],
         [ 2.4405e-01,  1.1619e-03,  3.0531e-01,  2.8206e-01],
         [ 3.1882e-01, -3.9534e-01, -1.1444e-01, -4.3760e-02],
         [ 1.3187e-01,  7.9257e-01, -2.2410e-01, -2.8045e-02]],

        [[-1.4559e+00,  3.3509e+00,  6.4862e-01, -2.1494e+00],
         [ 8.8416e-01,  1.2517e+00,  1.7419e-01, -9.0476e-02],
         [ 1.7910e+00, -2.0875e+00, -9.7189e-01,  1.3043e+00],
         [-1.0137e+00,  2.0690e+00,  2.2690e-01, -9.3167e-01]]])

Again, codes can run, but need to check correctness.

# Try to run training

In [23]:
import torch
from absl import app, flags
from stable_baselines3 import SAC, PPO
from envs.wahba import Wahba
from stable_baselines_utils import CustomSACPolicy, \
    CustomCNN

In [24]:
def main(argv):
    env = Wahba()
    device = torch.device('cpu')
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 'auto_0.1',
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 500, eval_freq = 100, n_eval_episodes = 100)

In [25]:
from torch import autograd
with autograd.detect_anomaly():
    main(None)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -7.3     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 219      |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -7.4     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 334      |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.76    |
| time/              |          |
|    episodes        | 12       |
|    fps             |

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.2     |
| time/              |          |
|    episodes        | 100      |
|    fps             | 792      |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------


/var/folders/kr/ks3dcr657632lx8hgb0jfh_w0000gn/T/ipykernel_10916/381243426.py:2: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():
/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/site-packages/stable_baselines3/common/buffers.py:220: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 4.82GB > 0.55GB
  warnings.warn(
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


tensor([[ 0.0282,  0.0451,  0.0034, -0.0394, -0.0057,  0.0364,  0.0276,  0.0125,
         -0.0045,  0.0236, -0.0181,  0.0141]])
sigma:  tensor([[0.0282, 0.0451, 0.0034]])
torch.Size([1, 3, 3])
tensor([[ 0.0285,  0.0453,  0.0035,  ...,  0.0233, -0.0180,  0.0146],
        [ 0.0281,  0.0455,  0.0038,  ...,  0.0237, -0.0182,  0.0144],
        [ 0.0277,  0.0452,  0.0037,  ...,  0.0241, -0.0181,  0.0141],
        ...,
        [ 0.0289,  0.0451,  0.0032,  ...,  0.0234, -0.0181,  0.0143],
        [ 0.0282,  0.0453,  0.0040,  ...,  0.0238, -0.0180,  0.0146],
        [ 0.0282,  0.0455,  0.0032,  ...,  0.0233, -0.0176,  0.0145]],
       grad_fn=<AddmmBackward0>)
sigma:  tensor([[0.0285, 0.0453, 0.0035],
        [0.0281, 0.0455, 0.0038],
        [0.0277, 0.0452, 0.0037],
        [0.0284, 0.0454, 0.0036],
        [0.0282, 0.0451, 0.0037],
        [0.0283, 0.0453, 0.0037],
        [0.0286, 0.0451, 0.0032],
        [0.0282, 0.0454, 0.0039],
        [0.0288, 0.0454, 0.0034],
        [0.0286, 0.0454, 0

tensor([[ 0.0285,  0.0453,  0.0035,  ...,  0.0233, -0.0180,  0.0146],
        [ 0.0281,  0.0455,  0.0038,  ...,  0.0237, -0.0182,  0.0144],
        [ 0.0277,  0.0452,  0.0037,  ...,  0.0241, -0.0181,  0.0141],
        ...,
        [ 0.0289,  0.0451,  0.0032,  ...,  0.0234, -0.0181,  0.0143],
        [ 0.0282,  0.0453,  0.0040,  ...,  0.0238, -0.0180,  0.0146],
        [ 0.0282,  0.0455,  0.0032,  ...,  0.0233, -0.0176,  0.0145]])
sigma:  tensor([[0.0285, 0.0453, 0.0035],
        [0.0281, 0.0455, 0.0038],
        [0.0277, 0.0452, 0.0037],
        [0.0284, 0.0454, 0.0036],
        [0.0282, 0.0451, 0.0037],
        [0.0283, 0.0453, 0.0037],
        [0.0286, 0.0451, 0.0032],
        [0.0282, 0.0454, 0.0039],
        [0.0288, 0.0454, 0.0034],
        [0.0286, 0.0454, 0.0035],
        [0.0281, 0.0453, 0.0040],
        [0.0282, 0.0453, 0.0040],
        [0.0277, 0.0456, 0.0036],
        [0.0280, 0.0452, 0.0040],
        [0.0283, 0.0455, 0.0035],
        [0.0282, 0.0455, 0.0040],
        [0.028

...

tensor([[45.7620, 45.0481, 43.4788,  ...,  2.1892, -8.0821, -1.4698],
        [47.3449, 46.6036, 44.9825,  ...,  2.2638, -8.3620, -1.5227],
        [50.5743, 49.7810, 48.0509,  ...,  2.4177, -8.9309, -1.6280],
        ...,
        [48.1052, 47.3520, 45.7046,  ...,  2.3001, -8.4953, -1.5475],
        [43.9420, 43.2563, 41.7485,  ...,  2.1021, -7.7619, -1.4110],
        [45.2263, 44.5207, 42.9690,  ...,  2.1639, -7.9880, -1.4527]])
sigma:  tensor([[45.7620, 45.0481, 43.4788],
        [47.3449, 46.6036, 44.9825],
        [50.5743, 49.7810, 48.0509],
        [50.1697, 49.3830, 47.6666],
        [47.0984, 46.3614, 44.7484],
        [43.2682, 42.5938, 41.1082],
        [45.5700, 44.8580, 43.2954],
        [44.2713, 43.5798, 42.0608],
        [44.6382, 43.9415, 42.4093],
        [47.0553, 46.3200, 44.7077],
        [42.4190, 41.7580, 40.3004],
        [47.0984, 46.3614, 44.7484],
        [43.8044, 43.1218, 41.6184],
        [44.9183, 44.2171, 42.6764],
        [49.6558, 48.8778, 47.1781],
   

tensor([[50.4787, 49.7866, 47.9161, -2.2741, -6.7827,  0.4379, -5.9095,  6.8910,
          6.0589,  2.3917, -8.8440, -1.6779]])
sigma:  tensor([[50.4787, 49.7866, 47.9161]])
torch.Size([1, 3, 3])
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -6.12    |
| time/              |          |
|    episodes        | 144      |
|    fps             | 1        |
|    time_elapsed    | 110      |
|    total_timesteps | 144      |
| train/             |          |
|    actor_loss      | 6.22     |
|    critic_loss     | 1.94     |
|    ent_coef        | 0.0986   |
|    ent_coef_loss   | -28.8    |
|    learning_rate   | 0.0003   |
|    n_updates       | 43       |
---------------------------------
tensor([[56.3638, 55.5863, 53.5023,  ...,  2.6688, -9.8738, -1.8776],
        [56.3054, 55.5287, 53.4466,  ...,  2.6661, -9.8637, -1.8757],
        [54.0733, 53.3286, 51.3275,  ...,  2.5609, -9.4735, -1.8005],
        ...,
   

tensor([[56.3638, 55.5863, 53.5023,  ...,  2.6688, -9.8738, -1.8776],
        [56.3054, 55.5287, 53.4466,  ...,  2.6661, -9.8637, -1.8757],
        [54.0733, 53.3286, 51.3275,  ...,  2.5609, -9.4735, -1.8005],
        ...,
        [55.2340, 54.4729, 52.4295,  ...,  2.6159, -9.6762, -1.8396],
        [51.2679, 50.5629, 48.6644,  ...,  2.4285, -8.9826, -1.7056],
        [56.0768, 55.3035, 53.2300,  ...,  2.6552, -9.8235, -1.8678]])
sigma:  tensor([[56.3638, 55.5863, 53.5023],
        [56.3054, 55.5287, 53.4466],
        [54.0733, 53.3286, 51.3275],
        [49.6316, 48.9506, 47.1111],
        [48.4473, 47.7832, 45.9863],
        [49.2031, 48.5274, 46.7035],
        [47.4863, 46.8360, 45.0746],
        [56.5796, 55.7989, 53.7071],
        [53.8568, 53.1155, 51.1227],
        [52.1077, 51.3914, 49.4620],
        [53.1254, 52.3936, 50.4280],
        [50.4991, 49.8064, 47.9347],
        [48.7986, 48.1297, 46.3198],
        [54.8903, 54.1343, 52.1038],
        [56.5414, 55.7613, 53.6708],
   

/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/site-packages/torch/autograd/__init__.py:154: UserWarning: Error detected in MulBackward0. Traceback of forward call that caused the error:
  File "/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/Users/weijiazeng/opt/anaconda3/envs/BPP/lib/python3.8/site-packa

RuntimeError: Function 'MulBackward0' returned nan values in its 1th output.

In [ ]:
%load_ext tensorboard
import tensorflow as tf
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

In [ ]:
%tensorboard --logdir ./sac

# Experiments for batch operations

In [ ]:
sigma = torch.ones(1, 3)
omiga = torch.normal(torch.zeros(1, 3), sigma)
omiga

In [ ]:
def transfer(omiga):
    omiga_0, omiga_1, omiga_2 = omiga[0], omiga[1], omiga[2]
    omiga_hat = torch.tensor([[0, -omiga_2, omiga_1],
                                [omiga_2, 0, -omiga_0],
                                [-omiga_1, omiga_0, 0]])
    return omiga_hat

In [ ]:
from functorch import vmap
batch_transfer = vmap(transfer)
batch_transfer(omiga)

In [ ]:
from liegroups.torch import SO3
C = SO3.exp(torch.Tensor([[1,2,3],
                          [0,0,0]]))
print(torch.Tensor([[1,2,3],
                          [0,0,0]]).size())
SO3.log(C)

In [ ]:
np.log(1)

# Question to ask: the original wahba problem action is (4,), in our case actions are (3,3).